In [13]:
import pandas as pd 
import numpy as np
import os
dataset_folder = '../dataset_new'

In [14]:
sensors = ['sensor2', 'sensor3', 'sensor4','sensor7','sensor8','sensor9','sensor11','sensor12','sensor13','sensor14',
           'sensor15','sensor17','sensor20', 'sensor21']

In [15]:
train = []
files = os.listdir(dataset_folder)
train_files = [file for file in files if file.startswith('scaled')]

for file in files: 
    path_to_file = dataset_folder + '/'+file
    df = pd.read_csv(path_to_file, index_col=0) #index_col = 0 perché sennò viene aggiunta la colonna unnamed. 
    train.append(df)

train[0].head()

,unit_number,time_cycle,op_setting1,op_setting2,op_setting3,RUL,sensor2,sensor3,sensor4,sensor6,...,sensor8,sensor9,sensor11,sensor12,sensor13,sensor14,sensor15,sensor17,sensor20,sensor21
0,1.0,1.0,-0.0005,0.0004,100.0,258.0,0.355972,0.370523,0.308580,1.0,...,0.623529,0.204233,0.348571,0.231279,0.642857,0.239116,0.647755,0.272727,0.559524,0.446331
1,1.0,2.0,0.0008,-0.0003,100.0,257.0,0.388759,0.399100,0.309360,1.0,...,0.647059,0.202431,0.308571,0.236882,0.654762,0.278567,0.685659,0.363636,0.488095,0.534836
2,1.0,3.0,-0.0014,-0.0002,100.0,256.0,0.313817,0.353298,0.445398,1.0,...,0.664706,0.241484,0.302857,0.217015,0.636905,0.264526,0.564462,0.272727,0.404762,0.458577
3,1.0,4.0,-0.0020,0.0001,100.0,255.0,0.487119,0.417107,0.237285,1.0,...,0.647059,0.215326,0.314286,0.240448,0.684524,0.245612,0.558909,0.363636,0.470238,0.391966
4,1.0,5.0,0.0016,0.0000,100.0,254.0,0.196721,0.476218,0.321217,1.0,...,0.670588,0.268799,0.262857,0.245033,0.654762,0.252109,0.556736,0.363636,0.577381,0.543371


La seguente funzione *process_targets* è la cosiddetta funzione **lineare a tratti** di cui parliamo estensivamente all'interno del readme ci permette dunque di calcolare il valore della RUL da assegnare ad ogni elemento del dataset. 
La funzione prende in input la lunghezza totale dei dati e la *"early_rul"* la quale rappresenta il valore massimo possibile di RUL (imponiamo ciò poiché come descritto dalla letteratura dovrebbe permettere alla rete di comprendere meglio quando il componente è in salute o meno), il quale poi viene decrementato in maniera lineare quando la *"data_length"* supera il valore di *early_rul*

In [16]:
def process_targets(data_length, early_rul = None):
    if early_rul == None:
        return np.arange(data_length-1, -1, -1)
    else:
        early_rul_duration = data_length - early_rul
        if early_rul_duration <= 0:
            return np.arange(data_length-1, -1, -1)
        else:
            return np.append(early_rul*np.ones(shape = (early_rul_duration,)), np.arange(early_rul-1, -1, -1))

*process_input_data_with_targets* è invece la funzione addetta a "sistemare" i valori all'interno delle loro proprie sequenze di *window_length* valori. 

In [17]:
def process_input_data_with_targets(input_data, target_data = None, window_length = 1, shift = 1):
    #Viene calcolato il numero di batch che saranno generati sulla base della grandezza dell'input 
    num_batches = int(np.floor((len(input_data) - window_length)/shift)) + 1
    
    #Si recupera il numero di features all'interno del dataframe
    num_features = input_data.shape[1]
    '''
    Qui, viene inizializzata una matrice output_data con valori NaN, che rappresenta i dati di output che verranno generati dalla funzione. 
    La matrice è inizialmente creata come una matrice 3D con dimensioni (num_batches, window_length, num_features) per contenere i batch di dati di input.
    '''
    output_data = np.repeat(np.nan, repeats = num_batches * window_length * num_features).reshape(num_batches, window_length,
                                                                                                  num_features)
    
    #Verfico che i labels siano stati forniti
    if target_data is None:
        
        #Iteriamo attraverso i batch e copiamo le finestre temporali corrispondenti dai dati di input input_data nella matrice output_data. 
        #L'output sarà quindi una matrice 3D con i batch di dati di input.

        for batch in range(num_batches):
            output_data[batch,:,:] = input_data[(0+shift*batch):(0+shift*batch+window_length),:]
        return output_data
    else:
        output_targets = np.repeat(np.nan, repeats = num_batches)
        #Nel caso in cui i dati siano forniti semplicemente facciamo la stessa cosa per i dati di target
        for batch in range(num_batches):
            output_data[batch,:,:] = input_data[(0+shift*batch):(0+shift*batch+window_length),:]
            output_targets[batch] = target_data[(shift*batch + (window_length-1))] #Differente perché i dati di target sono formattati in altra maniera.
        return output_data, output_targets

In [18]:
#Configurazione

window_length = 30 #Lunghezza delle sequenze
shift = 1
early_rul = 125 #100 è il valore utilizzato comunemente dallo stato dell'arte             
processed_train_data = []
processed_train_targets = []

FD0001 = train[0]
unit_number_col = FD0001['unit_number']
num_train_machines_FD0001 = len(FD0001['unit_number'].unique())
FD0001 = FD0001[sensors]

In [19]:
new_FD0001 = pd.concat([unit_number_col, FD0001], axis= 1 )

In [20]:
new_FD0001.head()

,unit_number,sensor2,sensor3,sensor4,sensor7,sensor8,sensor9,sensor11,sensor12,sensor13,sensor14,sensor15,sensor17,sensor20,sensor21
0,1.0,0.355972,0.370523,0.308580,0.208812,0.623529,0.204233,0.348571,0.231279,0.642857,0.239116,0.647755,0.272727,0.559524,0.446331
1,1.0,0.388759,0.399100,0.309360,0.236590,0.647059,0.202431,0.308571,0.236882,0.654762,0.278567,0.685659,0.363636,0.488095,0.534836
2,1.0,0.313817,0.353298,0.445398,0.230843,0.664706,0.241484,0.302857,0.217015,0.636905,0.264526,0.564462,0.272727,0.404762,0.458577
3,1.0,0.487119,0.417107,0.237285,0.268199,0.647059,0.215326,0.314286,0.240448,0.684524,0.245612,0.558909,0.363636,0.470238,0.391966
4,1.0,0.196721,0.476218,0.321217,0.245690,0.670588,0.268799,0.262857,0.245033,0.654762,0.252109,0.556736,0.363636,0.577381,0.543371


In [21]:

for i in np.arange(1, num_train_machines_FD0001 + 1):
    temp_train_data = new_FD0001[new_FD0001['unit_number'] == i].drop(columns = ['unit_number']).values
    
    # Verify if data of given window length can be extracted from training data
    if (len(temp_train_data) < window_length):
        print("Train engine {} doesn't have enough data for window_length of {}".format(i, window_length))
        raise AssertionError("Window length is larger than number of data points for some engines. "
                             "Try decreasing window length.")
        
    temp_train_targets = process_targets(data_length = temp_train_data.shape[0], early_rul = early_rul)
    data_for_a_machine, targets_for_a_machine = process_input_data_with_targets(temp_train_data, temp_train_targets, 
                                                                                window_length = window_length, shift = shift)
    
    processed_train_data.append(data_for_a_machine)
    processed_train_targets.append(targets_for_a_machine)

processed_train_data = np.concatenate(processed_train_data)
processed_train_targets = np.concatenate(processed_train_targets)

In [22]:

print("Processed trianing data shape: ", processed_train_data.shape)
print("Processed training ruls shape: ", processed_train_targets.shape)

Processed trianing data shape:  (21820, 30, 14)
Processed training ruls shape:  (21820,)


In [23]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(processed_train_data, processed_train_targets, test_size=0.2, random_state=42)

In [24]:
from keras import backend as K 
import tensorflow as tf 
import math 

@tf.function
def custom_loss2(y_true, y_pred):
    alpha = 0.2
    difference = y_pred - y_true
    squared_difference = tf.square(y_pred - y_true)
    
    # Calcola la loss per ciascun elemento
    loss = tf.where(difference < 0, 2 * alpha * squared_difference, 2 * (alpha + (1 - 2 * alpha)) * squared_difference)
    
    # Calcola la media delle loss
    return tf.reduce_mean(loss)

@tf.function
def custom_score2(y_true, y_pred):
    d_i = y_pred - y_true
    esponente = tf.where(d_i < 0, 1.0 / (d_i / 13.0), d_i / 10.0)
    sum = np.sum(np.where(esponente < 0, np.exp(-esponente/13)-1, np.exp(esponente/10)-1))
    return sum


def compute_s_score(rul_true, rul_pred):
    """
    Both rul_true and rul_pred should be 1D numpy arrays.
    """
    diff = rul_pred - rul_true
    return np.sum(np.where(diff < 0, np.exp(-diff/13)-1, np.exp(diff/10)-1))

2023-09-28 10:46:23.087744: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-28 10:46:24.659856: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda-11.2/lib64:/usr/local/cuda-11.2/extras/CUPTI/lib64:/home/aliquodfahriam/miniconda3/envs/tensorflowEnv/lib/
2023-09-28 10:46:24.660302: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/

In [25]:
from keras import regularizers
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Activation
from keras.layers import Dropout
from keras.layers import Flatten
from keras.optimizers import Adam


#Replica della rete neurale proposta come "small LSTM"
def createLSTMsmallModel(l1Nodes, l2Nodes, d1Nodes, d2Nodes, input_shape):
    # input layer
    lstm1 = LSTM(l1Nodes, input_shape=input_shape,activation='tanh', return_sequences=True, kernel_regularizer=regularizers.l2(0.1))
    
    lstm2 = LSTM(l2Nodes, return_sequences=True, activation='tanh', kernel_regularizer=regularizers.l2(0.1))    
    flatten = Flatten()
    dense1 = Dense(d1Nodes, activation='relu', kernel_regularizer=regularizers.l2(0.1))
    dense2 = Dense(d2Nodes, activation='relu', kernel_regularizer=regularizers.l2(0.1))
    
    # output layer
    outL = Dense(1, activation='relu', kernel_regularizer=regularizers.l2(0.1))
    
    #layers
    layers = [lstm1, lstm2,flatten,  dense1, dense2, outL]
    model = Sequential(layers)
    #Abbiamo aggiunto le nostre funzioni di loss e accuracy definite precedentemente

    optimizer = Adam(learning_rate=0.1)
    model.compile(optimizer=optimizer, loss=custom_loss2, metrics = [custom_score2] )
    return model


def createLSTMlargeModel(l1Nodes, l2Nodes, l3Nodes, l4Nodes, d1dense , d2dense, input_shape): 
    lstm1 = LSTM(l1Nodes, input_shape=input_shape, return_sequences=True, kernel_regularizer=regularizers.l2(0.1))
    d1 = Dropout(0.2)
    lstm2 = LSTM(l2Nodes, return_sequences=True, kernel_regularizer=regularizers.l2(0.1))
    d2 = Dropout(0.2)
    lstm3 = LSTM(l3Nodes, return_sequences=True, kernel_regularizer=regularizers.l2(0.1))
    d3 = Dropout(0.2)
    lstm4 = LSTM(l4Nodes, return_sequences=True, kernel_regularizer=regularizers.l2(0.1))
    d4 = Dropout(0.2)

    flatten = Flatten()

    dense1 = Dense(d1dense,activation='relu', kernel_regularizer=regularizers.l2(0.1))
    dense2 = Dense(d2dense,activation='relu', kernel_regularizer=regularizers.l2(0.1))
    output_dense = Dense(1,activation='relu', kernel_regularizer=regularizers.l2(0.1))

    layers = [lstm1,d1, lstm2,d2, lstm3, d3, lstm4, d4,flatten,  dense1, dense2, output_dense]
   
    optimizer = Adam(learning_rate=0.1)
    model = Sequential(layers)
    model.compile(optimizer=optimizer, loss=custom_loss2, metrics = [custom_score2])
    return model


In [26]:
LSTMlargeModel= createLSTMlargeModel(128, 64, 32, 16, 64, 32, (30,14))
LSTMsmallModel = createLSTMsmallModel(60, 30, 30, 15, (30, 14)) 

2023-09-28 10:46:28.199281: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-09-28 10:46:28.199377: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pop-os): /proc/driver/nvidia/version does not exist
2023-09-28 10:46:28.200586: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [27]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model, save_model

EPOCHS = 50 #100 / 80 / 150 

path_small = './models/LSTMsmall/FD0001'
path_large = './models/LSTMlarge/FD0001'


model_checkpoint = ModelCheckpoint(
    path_small,
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',

)


early_stopping = EarlyStopping(
    patience=5,
    min_delta=0.001,
    restore_best_weights=True,
)



In [28]:
# history_small = LSTMsmallModel.fit(
#     X_train, 
#     y_train, 
#     validation_data=(X_val, y_val), 
#     epochs = EPOCHS,
#     batch_size = 256,
#     use_multiprocessing =True, 
#     callbacks=[model_checkpoint, early_stopping])

Epoch 1/50
68/69 [============================>.] - ETA: 0s - loss: 2368.7158 - custom_score2: 110955760.0000
Epoch 1: val_loss improved from inf to 1746.49475, saving model to ./models/LSTMsmall/FD0001


INFO:tensorflow:Assets written to: ./models/LSTMsmall/FD0001/assets


INFO:tensorflow:Assets written to: ./models/LSTMsmall/FD0001/assets


69/69 [==============================] - 14s 147ms/step - loss: 2366.2920 - custom_score2: 109347712.0000 - val_loss: 1746.4948 - val_custom_score2: 5836.0674
Epoch 2/50
68/69 [============================>.] - ETA: 0s - loss: 1709.7483 - custom_score2: 6516.5425
Epoch 2: val_loss improved from 1746.49475 to 1653.49426, saving model to ./models/LSTMsmall/FD0001


INFO:tensorflow:Assets written to: ./models/LSTMsmall/FD0001/assets


...

INFO:tensorflow:Assets written to: ./models/LSTMsmall/FD0001/assets


69/69 [==============================] - 8s 114ms/step - loss: 161.5265 - custom_score2: 64.4005 - val_loss: 156.1916 - val_custom_score2: 31.5463
Epoch 32/50
69/69 [==============================] - ETA: 0s - loss: 155.5556 - custom_score2: 57.0017
Epoch 32: val_loss did not improve from 156.19156
69/69 [==============================] - 3s 48ms/step - loss: 155.5556 - custom_score2: 57.0017 - val_loss: 164.2293 - val_custom_score2: 160.0748
Epoch 33/50
69/69 [==============================] - ETA: 0s - loss: 156.5530 - custom_score2: 55.8323
Epoch 33: val_loss improved from 156.19156 to 146.14778, saving model to ./models/LSTMsmall/FD0001


INFO:tensorflow:Assets written to: ./models/LSTMsmall/FD0001/assets


INFO:tensorflow:Assets written to: ./models/LSTMsmall/FD0001/assets


69/69 [==============================] - 8s 121ms/step - loss: 156.5530 - custom_score2: 55.8323 - val_loss: 146.1478 - val_custom_score2: 55.7221
Epoch 34/50
69/69 [==============================] - ETA: 0s - loss: 164.1631 - custom_score2: 75.8162
Epoch 34: val_loss did not improve from 146.14778
69/69 [==============================] - 3s 50ms/step - loss: 164.1631 - custom_score2: 75.8162 - val_loss: 159.1275 - val_custom_score2: 48.0689
Epoch 35/50
68/69 [============================>.] - ETA: 0s - loss: 229.8910 - custom_score2: 186.3955
Epoch 35: val_loss did not improve from 146.14778
69/69 [==============================] - 3s 50ms/step - loss: 229.8740 - custom_score2: 184.7120 - val_loss: 425.0716 - val_custom_score2: -98.0729
Epoch 36/50
69/69 [==============================] - ETA: 0s - loss: 179.0030 - custom_score2: 78.5991
Epoch 36: val_loss did not improve from 146.14778
69/69 [==============================] - 4s 52ms/step - loss: 179.0030 - custom_score2: 78.5991 - v

In [29]:
model_checkpoint = ModelCheckpoint(
    path_large,
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    save_weights_only=False,
    mode='auto',

)
history_large = LSTMlargeModel.fit(
    X_train, 
    y_train, 
    validation_data=(X_val, y_val), 
    epochs = 100,
    batch_size = 256,
    use_multiprocessing =True, 
    callbacks=[model_checkpoint, early_stopping])

Epoch 1/100
546/546 [==============================] - ETA: 0s - loss: 4591.7383 - custom_score2: 704040635727872.0000
Epoch 1: val_loss improved from inf to 4116.91846, saving model to ./models/LSTMlarge/FD0001


INFO:tensorflow:Assets written to: ./models/LSTMlarge/FD0001/assets


INFO:tensorflow:Assets written to: ./models/LSTMlarge/FD0001/assets


546/546 [==============================] - 44s 73ms/step - loss: 4591.7383 - custom_score2: 704040635727872.0000 - val_loss: 4116.9185 - val_custom_score2: -6.4390
Epoch 2/100
546/546 [==============================] - ETA: 0s - loss: 4087.1772 - custom_score2: -6.4870
Epoch 2: val_loss improved from 4116.91846 to 4013.11963, saving model to ./models/LSTMlarge/FD0001


INFO:tensorflow:Assets written to: ./models/LSTMlarge/FD0001/assets


...

INFO:tensorflow:Assets written to: ./models/LSTMlarge/FD0001/assets


546/546 [==============================] - 41s 75ms/step - loss: 3855.2190 - custom_score2: -6.4870 - val_loss: 3824.6763 - val_custom_score2: -6.4390
Epoch 23/100
546/546 [==============================] - ETA: 0s - loss: 3855.2200 - custom_score2: -6.4870
Epoch 23: val_loss improved from 3824.67627 to 3824.67603, saving model to ./models/LSTMlarge/FD0001


INFO:tensorflow:Assets written to: ./models/LSTMlarge/FD0001/assets


INFO:tensorflow:Assets written to: ./models/LSTMlarge/FD0001/assets


546/546 [==============================] - 41s 75ms/step - loss: 3855.2200 - custom_score2: -6.4870 - val_loss: 3824.6760 - val_custom_score2: -6.4390
Epoch 24/100
546/546 [==============================] - ETA: 0s - loss: 3855.2190 - custom_score2: -6.4869
Epoch 24: val_loss did not improve from 3824.67603
546/546 [==============================] - 31s 56ms/step - loss: 3855.2190 - custom_score2: -6.4869 - val_loss: 3824.6760 - val_custom_score2: -6.4390
Epoch 25/100
546/546 [==============================] - ETA: 0s - loss: 3855.2195 - custom_score2: -6.4870
Epoch 25: val_loss did not improve from 3824.67603
546/546 [==============================] - 32s 58ms/step - loss: 3855.2195 - custom_score2: -6.4870 - val_loss: 3824.6760 - val_custom_score2: -6.4390
Epoch 26/100
546/546 [==============================] - ETA: 0s - loss: 3855.2183 - custom_score2: -6.4870
Epoch 26: val_loss did not improve from 3824.67603
546/546 [==============================] - 32s 58ms/step - loss: 3855.218